In [3]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [4]:
!imkdir -p ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

'imkdir' is not recognized as an internal or external command,
operable program or batch file.


'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!kaggle datasets download -d uciml/sms-spam-collection-dataset

Dataset URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
License(s): unknown
sms-spam-collection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
from zipfile import ZipFile
dataset = 'sms-spam-collection-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print(' dataset extracted :)')

 dataset extracted :)


In [21]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2

In [8]:
# Load the dataset
df = pd.read_csv('spam.csv',encoding='ISO-8859-1')

#NO. of rows and cols
df.shape


(5572, 5)

In [9]:
# data preprocessing
df.info()
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)
df.rename(columns={'v1':'label','v2':'message'},inplace=True)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df.isnull().sum()
df.duplicated().sum()
df = df.drop_duplicates(keep='first')
df.shape

(5169, 2)

In [11]:
df['label'].value_counts()
df['label'].replace({'ham':0,'spam':1},inplace=True)
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HARSHITA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HARSHITA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    ps = PorterStemmer()
    stemmed_text = [ps.stem(word) for word in filtered_text]
    return ' '.join(stemmed_text)
df['message'] = df['message'].apply(preprocess_text)



In [14]:
#splitting the data
X = df['message']
y = df['label']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(X.shape,X_train.shape,X_test.shape)
print(X_train_tfidf.shape,X_test_tfidf.shape)

(5169,) (4135,) (1034,)
(4135, 6230) (1034, 6230)


In [15]:
selector = SelectKBest(chi2, k=6230)
X_train_selected = selector.fit_transform(X_train_tfidf, y_train)
X_test_selected = selector.transform(X_test_tfidf)

# Train Naive Bayes classifier
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

# Predict on test data
y_pred_nb = nb.predict(X_test_tfidf)

# Evaluate Naive Bayes classifier
accuracy_nb = accuracy_score(y_test, y_pred_nb)

# Set pos_label to 1 to match your target labels
precision_nb = precision_score(y_test, y_pred_nb, pos_label=1)  
recall_nb = recall_score(y_test, y_pred_nb, pos_label=1)
f1_nb = f1_score(y_test, y_pred_nb, pos_label=1)
train_confusion = confusion_matrix(y_train, nb.predict(X_train_tfidf)) # Calculate confusion matrix for training data
test_confusion = confusion_matrix(y_test, y_pred_nb) # Calculate confusion matrix for testing data

print("Naive Bayes - Accuracy:", accuracy_nb, "Precision:", precision_nb, "Recall:", recall_nb, "F1 Score:", f1_nb)
print("Training Confusion Matrix:")
print(train_confusion)

Naive Bayes - Accuracy: 0.9661508704061895 Precision: 1.0 Recall: 0.7586206896551724 F1 Score: 0.8627450980392157
Training Confusion Matrix:
[[3627    0]
 [  96  412]]


In [16]:
# Train Logistic Regression classifier
lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)

# Predict on test data
y_pred_lr = lr.predict(X_test_tfidf)

# Evaluate Logistic Regression classifier
accuracy_lr = accuracy_score(y_test, y_pred_lr)
# Assuming 1 represents 'spam' in your dataset
precision_lr = precision_score(y_test, y_pred_lr, pos_label=1)  
recall_lr = recall_score(y_test, y_pred_lr, pos_label=1)  
f1_lr = f1_score(y_test, y_pred_lr, pos_label=1)  
print("Logistic Regression - Accuracy:", accuracy_lr, "Precision:", precision_lr, "Recall:", recall_lr, "F1 Score:", f1_lr)

Logistic Regression - Accuracy: 0.9671179883945842 Precision: 0.9826086956521739 Recall: 0.7793103448275862 F1 Score: 0.8692307692307693


In [17]:
# Train Support Vector Machine classifier
svm = SVC()
svm.fit(X_train_tfidf, y_train)

# Predict on test data
y_pred_svm = svm.predict(X_test_tfidf)

# Evaluate Support Vector Machine classifier
accuracy_svm = accuracy_score(y_test, y_pred_svm)
# Set pos_label to 1 to be consistent with previous evaluations 
precision_svm = precision_score(y_test, y_pred_svm, pos_label=1)  
recall_svm = recall_score(y_test, y_pred_svm, pos_label=1)
f1_svm = f1_score(y_test, y_pred_svm, pos_label=1)
print("Support Vector Machine - Accuracy:", accuracy_svm, "Precision:", precision_svm, "Recall:", recall_svm, "F1 Score:", f1_svm)

Support Vector Machine - Accuracy: 0.9806576402321083 Precision: 1.0 Recall: 0.8620689655172413 F1 Score: 0.9259259259259259


In [18]:
models = {
    "Naive Bayes" : MultinomialNB(),
    "Logistic Regression" : LogisticRegression(),
    "Support Vector Machine" : SVC()
}
results = []
for model_name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    train_accuracy = accuracy_score(y_train, model.predict(X_train_tfidf))
    test_accuracy = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred, pos_label=1)
    test_recall = recall_score(y_test, y_pred, pos_label=1)
    test_f1 = f1_score(y_test, y_pred, pos_label=1)
    train_confusion = confusion_matrix(y_train, model.predict(X_train_tfidf))
    test_confusion = confusion_matrix(y_test, y_pred)
  
    results.append({
        "Model": model_name,
        "Train Accuracy": train_accuracy,
        "Test Accuracy": test_accuracy,
        "Test Precision": test_precision,
        "Test Recall": test_recall,
        "Test F1 Score": test_f1
    })
results_df = pd.DataFrame(results)
results_df

,Model,Train Accuracy,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Naive Bayes,0.976784,0.966151,1.000000,0.758621,0.862745
1,Logistic Regression,0.963482,0.967118,0.982609,0.779310,0.869231
2,Support Vector Machine,0.997582,0.980658,1.000000,0.862069,0.925926


In [19]:
import pickle

# Assume we have a trained model and a vectorizer
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

# Save the model to a file
with open('spam_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

# Save the vectorizer to a file
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [20]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
